In [11]:
import django

In [12]:
django.setup()


In [3]:
from transformers.models import Transformation

Transformation.objects.all()

<QuerySet [<Transformation: SliceLineTransformer of Initial Stack of ROI created at 1569080922.371076 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_chNeuN647.lif>, <Transformation: MaskingRevamper of SliceLineTransformer of Initial Stack of ROI created at 1569080922.371076 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_chNeuN647.lif>, <Transformation: SliceLineTransformer of Initial Stack of ROI created at 1569080933.30008 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_chNeuN647.lif>, <Transformation: MaskingRevamper of SliceLineTransformer of Initial Stack of ROI created at 1569080933.30008 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_chNeuN647.lif>, <Transformation: SliceLineTransformer of Initial Stack of ROI created at 1569081020.780625 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_chNeuN647.lif>, <Transformation: MaskingRevamper of SliceLineTransformer of Initial Stack of ROI created at 1569081020.780625 on Sample ratM1_p15_T6_OT8_3.2_gpSynpo488_rbb4S564_ch

In [13]:
query = """{
  clusterdata(sample_Experiment_Name_Icontains: "p15") {
    roi {
      id
    }
    id
    nodeid
    sample {
      id
      name
    }
    clusterareapixels
    clusternumber
    clusterarea
  }
}"""


In [14]:
from gql import schema



In [15]:
result = schema.schema.execute(query)



In [16]:
resultdict = result.to_dict()

In [17]:
import json
resultjson = json.dumps(resultdict)



In [18]:
from pandas.io.json import json_normalize
pddataframe = json_normalize(resultdict["data"]["clusterdata"])
pddataframe.describe()

,clusterareapixels,clusternumber,clusterarea
count,10.000000,10.000000,10.000000
mean,43.500000,3.000000,2.516055
std,38.283301,2.211083,2.214319
min,0.000000,0.000000,0.000000
25%,9.250000,0.750000,0.535023
50%,40.000000,4.000000,2.313614
75%,66.500000,4.000000,3.846383
max,120.000000,6.000000,6.940841


In [20]:
import pandas_profiling
report = pddataframe.profile_report(style={'full_width':True})
report.
